In [73]:
#svd where test set = validation set

import json
from pathlib import Path
from sklearn.svm import SVR
from sentence_transformers import SentenceTransformer
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback




path_load_embeddings_training = "embeddings_training_All"



print("begin")

def flatten(list_of_list):
    return [item for sublist in list_of_list for item in sublist]

path_to_training = Path("training")
path_to_test = Path("training")  #be careful it's changed to training here

#####
# training and test sets of transcription ids
#####



# #small 1 (only to test code)
# training_set = ['ES2002']
# training_set = flatten([[m_id+s_id for s_id in 'ab'] for m_id in training_set])



# #small 2
# training_set = ['ES2002', 'ES2005', 'ES2006', 'IS1000', 'IS1001', 'IS1002', 'TS3005', 'TS3008']
# training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
# training_set.remove('IS1002a')



# #small 2bis
# training_set = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003' ,'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010']
# training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
# training_set.remove('IS1002a')


# #small 3
# training_set = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011']
# training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
# training_set.remove('IS1002a')
# training_set.remove('IS1005d')


## All
training_set = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2013', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1004', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011', 'TS3012']
training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
training_set.remove('IS1002a')
training_set.remove('IS1005d')
training_set.remove('TS3012c')





########################################################### Training #######################################################################

print("Loading training data")

y_training = []
with open("training_labels.json", "r") as file:
    training_labels = json.load(file)

for transcription_id in training_set:
    y_training += training_labels[transcription_id]




# text_SVD: utterances are embedded with SentenceTransformer, then train a support vector regression model.
X_training = json.load(open(path_load_embeddings_training + '/X_training.json', 'r', encoding='utf-8'))
X_training_edges = json.load(open(path_load_embeddings_training + '/X_training_edges.json', 'r', encoding='utf-8'))
X_training_speaker = json.load(open(path_load_embeddings_training + '/X_training_speaker.json', 'r', encoding='utf-8'))
X_training_previous = json.load(open(path_load_embeddings_training + '/X_training_previous.json', 'r', encoding='utf-8'))




begin
Loading training data


In [74]:
path_load_embeddings_test = "embeddings_test_Small2bis"
########################################################## Validation Data #######################################################################
print("Loading test data")


path_to_training = Path("training")
path_to_test = Path("training")  #be careful it's changed to training here

#####
# training and test sets of transcription ids
#####



# #small 1 (only to test code)
# test_set = ['ES2013']
# test_set = flatten([[m_id+s_id for s_id in 'ab'] for m_id in test_set])



# #small 2
# test_set = ['ES2013', 'IS1004', 'TS3012']
# test_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in test_set])
# test_set.remove('TS3012c')


#small 2 bis
test_set = ['ES2012', 'ES2013', 'IS1004','IS1005', 'TS3011', 'TS3012']
test_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in test_set])
test_set.remove('TS3012c')
test_set.remove('IS1005d')


# text_SVD: utterances are embedded with SentenceTransformer, then train a support vector regression model.
bert = SentenceTransformer('all-MiniLM-L6-v2')



y_test = []
with open("training_labels.json", "r") as file:
    training_labels = json.load(file)


for transcription_id in test_set:
    y_test += training_labels[transcription_id]





X_test = json.load(open(path_load_embeddings_test + '/X_test.json', 'r', encoding='utf-8'))
X_test_edges = json.load(open(path_load_embeddings_test + '/X_test_edges.json', 'r', encoding='utf-8'))
X_test_speaker = json.load(open(path_load_embeddings_test + '/X_test_speaker.json', 'r', encoding='utf-8'))
X_test_previous = json.load(open(path_load_embeddings_test + '/X_test_previous.json', 'r', encoding='utf-8'))

Loading test data


In [75]:

########################################################## Model #######################################################################

X_train_text = X_training 
X_train_title = X_training_edges 
X_train_link = X_training_speaker 
X_train_prev_sentence = X_training_previous 
y_train = y_training


X_train_text = np.array(X_train_text)
X_train_title = np.array(X_train_title)
X_train_link = np.array(X_train_link)
X_train_prev_sentence = np.array(X_train_prev_sentence)
y_train = np.array(y_train)





from tensorflow.keras.layers import Dropout

# Supposons que vous avez déjà vos embeddings pour chaque catégorie
embedding_dim = X_train_text.shape[1]
num_categories = 4  # Nombre de catégories d'embedding

# Entrées
input_text_embedding = layers.Input(shape=(embedding_dim,), name='text_embedding')
input_title_embedding = layers.Input(shape=(embedding_dim,), name='title_embedding')
input_link_embedding = layers.Input(shape=(embedding_dim,), name='link_embedding')
input_prev_sentence_embedding = layers.Input(shape=(embedding_dim,), name='prev_sentence_embedding')

# Concaténation des embeddings
concatenated = layers.concatenate([input_text_embedding, input_title_embedding, input_link_embedding, input_prev_sentence_embedding])

# Ajout de couches denses avec dropout
dense1 = layers.Dense(1536, activation='relu')(concatenated)
dropout1 = Dropout(0.6)(dense1)  # Ajout d'une couche de dropout avec un taux de dropout de 0.5
dense2 = layers.Dense(1024, activation='relu')(dropout1)
dropout2 = Dropout(0.4)(dense2)  # Ajout d'une couche de dropout avec un taux de dropout de 0.3
dense3 = layers.Dense(512, activation='relu')(dropout2)
dropout3= Dropout(0.3)(dense3)
dense4 = layers.Dense(256, activation='relu')(dropout3)
dropout4= Dropout(0.1)(dense4)
dense5 = layers.Dense(128, activation='relu')(dropout4)

# Couche de sortie
output = layers.Dense(1, activation='sigmoid', name='output')(dense5)

# Création du modèle
model = models.Model(inputs=[input_text_embedding, input_title_embedding, input_link_embedding, input_prev_sentence_embedding], outputs=output)

# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Afficher le résumé du modèle
model.summary()


########################################################## Fit #######################################################################
X_train_text = X_training 
X_train_title = X_training_edges 
X_train_link = X_training_speaker 
X_train_prev_sentence = X_training_previous 
y_train = y_training


X_val_text = X_test 
X_val_title = X_test_edges 
X_val_link = X_test_speaker 
X_val_prev_sentence = X_test_previous 
y_val = y_test





# Convert your data to numpy arrays
X_train_text = np.array(X_train_text)
X_train_title = np.array(X_train_title)
X_train_link = np.array(X_train_link)
X_train_prev_sentence = np.array(X_train_prev_sentence)
y_train = np.array(y_train)

X_val_text = np.array(X_val_text)
X_val_title = np.array(X_val_title)
X_val_link = np.array(X_val_link)
X_val_prev_sentence = np.array(X_val_prev_sentence)
y_val = np.array(y_val)




# checkpoint_path = "NN_weights.{epoch:02d}-{val_loss:.2f}.h5"

# # Créez un callback ModelCheckpoint pour sauvegarder le meilleur modèle
# checkpoint_callback = ModelCheckpoint(
#     filepath=checkpoint_path,
#     save_best_only=True,  # Ne sauvegarde que le meilleur modèle basé sur la métrique de validation
#     monitor='val_loss',   # Métrique à surveiller
#     mode='min',           # Mode "min" signifie que nous cherchons à minimiser la métrique de validation
#     verbose=1
# )




Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text_embedding (InputLayer  [(None, 384)]                0         []                            
 )                                                                                                
                                                                                                  
 title_embedding (InputLaye  [(None, 384)]                0         []                            
 r)                                                                                               
                                                                                                  
 link_embedding (InputLayer  [(None, 384)]                0         []                            
 )                                                                                          

Small 2

Epoch 1/40
255/255 [==============================] - 1s 3ms/steposs:
Epoch 1 - Validation F1 Score: 0.4555
797/797 [==============================] - 12s 15ms/step - loss: 0.0410 - accuracy: 0.9850 - val_loss: 1.1170 - val_accuracy: 0.8141
Epoch 2/40
255/255 [==============================] - 1s 3ms/stepos
Epoch 2 - Validation F1 Score: 0.4571
797/797 [==============================] - 12s 15ms/step - loss: 0.0429 - accuracy: 0.9844 - val_loss: 0.9899 - val_accuracy: 0.8083
Epoch 3/40
255/255 [==============================] - 1s 3ms/step
Epoch 3 - Validation F1 Score: 0.4354
797/797 [==============================] - 12s 15ms/step - loss: 0.0342 - accuracy: 0.9872 - val_loss: 0.9956 - val_accuracy: 0.8118
Epoch 4/40
255/255 [==============================] - 1s 3ms/step
Epoch 4 - Validation F1 Score: 0.4172
797/797 [==============================] - 13s 16ms/step - loss: 0.0334 - accuracy: 0.9881 - val_loss: 1.0291 - val_accuracy: 0.8155
Epoch 5/40
255/255 [==============================] - 1s 3ms/stepos
Epoch 5 - Validation F1 Score: 0.4453
797/797 [==============================] - 12s 15ms/step - loss: 0.0334 - accuracy: 0.9878 - val_loss: 0.9995 - val_accuracy: 0.7947
Epoch 6/40
255/255 [==============================] - 1s 3ms/stepos
Epoch 6 - Validation F1 Score: 0.4697
797/797 [==============================] - 12s 15ms/step - loss: 0.0252 - accuracy: 0.9910 - val_loss: 1.3875 - val_accuracy: 0.7978
Epoch 7/40
255/255 [==============================] - 1s 2ms/steposs: 0
Epoch 7 - Validation F1 Score: 0.4460
797/797 [==============================] - 12s 15ms/step - loss: 0.0313 - accuracy: 0.9890 - val_loss: 0.9889 - val_accuracy: 0.8106
Epoch 8/40
255/255 [==============================] - 1s 2ms/steposs:
Epoch 8 - Validation F1 Score: 0.4309
797/797 [==============================] - 12s 15ms/step - loss: 0.0239 - accuracy: 0.9920 - val_loss: 1.1914 - val_accuracy: 0.8206
Epoch 9/40
255/255 [==============================] - 1s 3ms/step
Epoch 9 - Validation F1 Score: 0.4616
797/797 [==============================] - 12s 15ms/step - loss: 0.0259 - accuracy: 0.9900 - val_loss: 1.2263 - val_accuracy: 0.7989
Epoch 10/40
255/255 [==============================] - 1s 3ms/steposs:
Epoch 10 - Validation F1 Score: 0.4696
797/797 [==============================] - 12s 15ms/step - loss: 0.0219 - accuracy: 0.9925 - val_loss: 1.2777 - val_accuracy: 0.8113
Epoch 11/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 11 - Validation F1 Score: 0.4527
797/797 [==============================] - 11s 14ms/step - loss: 0.0214 - accuracy: 0.9923 - val_loss: 1.3018 - val_accuracy: 0.8070
Epoch 12/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 12 - Validation F1 Score: 0.4396
797/797 [==============================] - 11s 14ms/step - loss: 0.0233 - accuracy: 0.9924 - val_loss: 1.1128 - val_accuracy: 0.8181
Epoch 13/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 13 - Validation F1 Score: 0.4910
797/797 [==============================] - 11s 14ms/step - loss: 0.0207 - accuracy: 0.9927 - val_loss: 1.2061 - val_accuracy: 0.8021
Epoch 14/40
255/255 [==============================] - 1s 2ms/steposs: 0.019
Epoch 14 - Validation F1 Score: 0.4692
797/797 [==============================] - 11s 14ms/step - loss: 0.0197 - accuracy: 0.9936 - val_loss: 1.0904 - val_accuracy: 0.8110
Epoch 15/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 15 - Validation F1 Score: 0.4569
797/797 [==============================] - 11s 14ms/step - loss: 0.0165 - accuracy: 0.9944 - val_loss: 1.3640 - val_accuracy: 0.8117
Epoch 16/40
255/255 [==============================] - 1s 2ms/steposs: 0.018
Epoch 16 - Validation F1 Score: 0.4769
797/797 [==============================] - 11s 14ms/step - loss: 0.0190 - accuracy: 0.9933 - val_loss: 1.2866 - val_accuracy: 0.8151
Epoch 17/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 17 - Validation F1 Score: 0.4758
797/797 [==============================] - 11s 14ms/step - loss: 0.0171 - accuracy: 0.9937 - val_loss: 1.4121 - val_accuracy: 0.7991
Epoch 18/40
255/255 [==============================] - 1s 2ms/steposs: 0.018
Epoch 18 - Validation F1 Score: 0.4749
797/797 [==============================] - 11s 14ms/step - loss: 0.0185 - accuracy: 0.9936 - val_loss: 1.4167 - val_accuracy: 0.7862
Epoch 19/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 19 - Validation F1 Score: 0.4368
797/797 [==============================] - 11s 14ms/step - loss: 0.0164 - accuracy: 0.9943 - val_loss: 1.5132 - val_accuracy: 0.8135
Epoch 20/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 20 - Validation F1 Score: 0.4491
797/797 [==============================] - 11s 14ms/step - loss: 0.0169 - accuracy: 0.9939 - val_loss: 1.3899 - val_accuracy: 0.8216
Epoch 21/40
255/255 [==============================] - 1s 2ms/steposs: 0
Epoch 21 - Validation F1 Score: 0.4654
797/797 [==============================] - 12s 14ms/step - loss: 0.0179 - accuracy: 0.9946 - val_loss: 1.4007 - val_accuracy: 0.8005
Epoch 22/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 22 - Validation F1 Score: 0.4676
797/797 [==============================] - 11s 14ms/step - loss: 0.0165 - accuracy: 0.9942 - val_loss: 1.2304 - val_accuracy: 0.8140
Epoch 23/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 23 - Validation F1 Score: 0.4786
797/797 [==============================] - 11s 14ms/step - loss: 0.0150 - accuracy: 0.9951 - val_loss: 1.3174 - val_accuracy: 0.8040
Epoch 24/40
255/255 [==============================] - 1s 2ms/steposs: 0.014
Epoch 24 - Validation F1 Score: 0.4692
797/797 [==============================] - 11s 14ms/step - loss: 0.0149 - accuracy: 0.9955 - val_loss: 1.2242 - val_accuracy: 0.8018
Epoch 25/40
255/255 [==============================] - 1s 2ms/steposs: 0
Epoch 25 - Validation F1 Score: 0.4239
797/797 [==============================] - 11s 14ms/step - loss: 0.0151 - accuracy: 0.9950 - val_loss: 1.1927 - val_accuracy: 0.8235
Epoch 26/40
255/255 [==============================] - 0s 2ms/steposs: 0.011
Epoch 26 - Validation F1 Score: 0.4686
797/797 [==============================] - 11s 14ms/step - loss: 0.0113 - accuracy: 0.9961 - val_loss: 1.6025 - val_accuracy: 0.8035
Epoch 27/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 27 - Validation F1 Score: 0.4650
797/797 [==============================] - 11s 14ms/step - loss: 0.0150 - accuracy: 0.9950 - val_loss: 1.2499 - val_accuracy: 0.8101
Epoch 28/40
255/255 [==============================] - 1s 2ms/steposs: 0
Epoch 28 - Validation F1 Score: 0.4855
797/797 [==============================] - 11s 14ms/step - loss: 0.0134 - accuracy: 0.9956 - val_loss: 1.4853 - val_accuracy: 0.7990
Epoch 29/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 29 - Validation F1 Score: 0.4423
797/797 [==============================] - 11s 14ms/step - loss: 0.0115 - accuracy: 0.9959 - val_loss: 1.3253 - val_accuracy: 0.8104
Epoch 30/40
255/255 [==============================] - 1s 2ms/steposs: 0.012
Epoch 30 - Validation F1 Score: 0.4644
797/797 [==============================] - 11s 14ms/step - loss: 0.0128 - accuracy: 0.9960 - val_loss: 1.2877 - val_accuracy: 0.8085
Epoch 31/40
255/255 [==============================] - 1s 2ms/steposs: 0.012
Epoch 31 - Validation F1 Score: 0.4590
797/797 [==============================] - 11s 14ms/step - loss: 0.0129 - accuracy: 0.9960 - val_loss: 1.1946 - val_accuracy: 0.8086
Epoch 32/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 32 - Validation F1 Score: 0.4280
797/797 [==============================] - 11s 14ms/step - loss: 0.0132 - accuracy: 0.9957 - val_loss: 1.0845 - val_accuracy: 0.8142
Epoch 33/40
255/255 [==============================] - 1s 2ms/steposs: 0.011
Epoch 33 - Validation F1 Score: 0.4421
797/797 [==============================] - 11s 14ms/step - loss: 0.0112 - accuracy: 0.9964 - val_loss: 1.4593 - val_accuracy: 0.8146
Epoch 34/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 34 - Validation F1 Score: 0.4373
797/797 [==============================] - 11s 14ms/step - loss: 0.0134 - accuracy: 0.9962 - val_loss: 1.2320 - val_accuracy: 0.8123
Epoch 35/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 35 - Validation F1 Score: 0.4436
797/797 [==============================] - 11s 14ms/step - loss: 0.0108 - accuracy: 0.9964 - val_loss: 1.2117 - val_accuracy: 0.8167
Epoch 36/40
255/255 [==============================] - 1s 2ms/steposs: 0.012
Epoch 36 - Validation F1 Score: 0.4566
797/797 [==============================] - 11s 14ms/step - loss: 0.0123 - accuracy: 0.9958 - val_loss: 1.3459 - val_accuracy: 0.8123
Epoch 37/40
255/255 [==============================] - 0s 2ms/steposs: 0.008
Epoch 37 - Validation F1 Score: 0.4726
797/797 [==============================] - 11s 14ms/step - loss: 0.0084 - accuracy: 0.9971 - val_loss: 1.7617 - val_accuracy: 0.8094
Epoch 38/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 38 - Validation F1 Score: 0.4215
797/797 [==============================] - 11s 14ms/step - loss: 0.0121 - accuracy: 0.9964 - val_loss: 1.3944 - val_accuracy: 0.8142
Epoch 39/40
255/255 [==============================] - 1s 2ms/steposs: 0.0
Epoch 39 - Validation F1 Score: 0.4795
797/797 [==============================] - 11s 14ms/step - loss: 0.0100 - accuracy: 0.9968 - val_loss: 1.2954 - val_accuracy: 0.7952
Epoch 40/40
255/255 [==============================] - 1s 2ms/steposs: 0.012
Epoch 40 - Validation F1 Score: 0.4612
797/797 [==============================] - 11s 14ms/step - loss: 0.0129 - accuracy: 0.9960 - val_loss: 1.4149 - val_accuracy: 0.7867

In [76]:

class F1ScoreCallback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        # Obtenez les prédictions sur l'ensemble de validation
        val_predictions = self.model.predict(self.validation_data[0])
        val_predictions_classes = (val_predictions > 0.35).astype(int)

        # Calcul du F1 score
        val_f1 = f1_score(self.validation_data[1], val_predictions_classes)

        # Affichage du F1 score
        print(f'Epoch {epoch + 1} - Validation F1 Score: {val_f1:.4f}')



nb_epochs = 8
# Entraînez le modèle avec le callback ModelCheckpoint
history = model.fit(
    [X_train_text, X_train_title, X_train_link, X_train_prev_sentence],
    y_train,
    epochs=nb_epochs,
    batch_size=32,
    validation_data=([X_val_text, X_val_title, X_val_link, X_val_prev_sentence], y_val),
    callbacks=[F1ScoreCallback(validation_data=([X_val_text, X_val_title, X_val_link, X_val_prev_sentence], y_val))]  
)



model.save_weights('NN_small3_epoch' + str(nb_epochs) + '.h5')





Epoch 1/8
482/482 [==============================] - 2s 4ms/step
Epoch 1 - Validation F1 Score: 0.5610
2270/2270 [==============================] - 57s 25ms/step - loss: 0.3527 - accuracy: 0.8249 - val_loss: 0.3276 - val_accuracy: 0.8354
Epoch 2/8
482/482 [==============================] - 2s 4ms/step
Epoch 2 - Validation F1 Score: 0.5694
2270/2270 [==============================] - 56s 25ms/step - loss: 0.3383 - accuracy: 0.8332 - val_loss: 0.3256 - val_accuracy: 0.8444
Epoch 3/8
482/482 [==============================] - 2s 4ms/step
Epoch 3 - Validation F1 Score: 0.5718
2270/2270 [==============================] - 56s 25ms/step - loss: 0.3320 - accuracy: 0.8376 - val_loss: 0.3188 - val_accuracy: 0.8402
Epoch 4/8
482/482 [==============================] - 2s 4ms/step
Epoch 4 - Validation F1 Score: 0.5808
2270/2270 [==============================] - 56s 25ms/step - loss: 0.3291 - accuracy: 0.8381 - val_loss: 0.3189 - val_accuracy: 0.8479
Epoch 5/8
482/482 [=============================

In [16]:
print("starting predictions")
predictions = model.predict([X_val_text, X_val_title, X_val_link, X_val_prev_sentence])
binary_predictions = (predictions > 0.43).astype(int)



# for i in range(len(binary_predictions)):
#     print(f"Exemple {i + 1} - Prédiction : {binary_predictions[i]}, Vraie étiquette : {y_test[i]}")

count10 = 0
for i in range(len(binary_predictions)):
    if(predictions[i]> 0.10):
        print(f"Exemple {i + 1} - Prédiction : {predictions[i]}, Vraie étiquette : {y_test[i]}")
        count10 +=1
print(count10/len(binary_predictions))

# Calcul du F1 score
f1 = f1_score(binary_predictions, y_test)

# Affichage du F1 score
print(f'F1 Score on Test Data: {f1}')

starting predictions
255/255 [==============================] - 1s 2ms/step
Exemple 13 - Prédiction : [0.8673351], Vraie étiquette : 1
Exemple 14 - Prédiction : [0.9413638], Vraie étiquette : 1
Exemple 15 - Prédiction : [0.72207725], Vraie étiquette : 1
Exemple 16 - Prédiction : [0.9334022], Vraie étiquette : 1
Exemple 17 - Prédiction : [0.26690158], Vraie étiquette : 1
Exemple 19 - Prédiction : [0.84890527], Vraie étiquette : 1
Exemple 20 - Prédiction : [0.9632832], Vraie étiquette : 1
Exemple 22 - Prédiction : [0.36929905], Vraie étiquette : 1
Exemple 25 - Prédiction : [0.7836134], Vraie étiquette : 1
Exemple 28 - Prédiction : [0.9776928], Vraie étiquette : 1
Exemple 31 - Prédiction : [0.60502094], Vraie étiquette : 1
Exemple 32 - Prédiction : [0.71131533], Vraie étiquette : 1
Exemple 36 - Prédiction : [0.8694176], Vraie étiquette : 1
Exemple 39 - Prédiction : [0.6082127], Vraie étiquette : 1
Exemple 40 - Prédiction : [0.23187007], Vraie étiquette : 0
Exemple 43 - Prédiction : [0.673

In [77]:
predictions = model.predict([X_val_text, X_val_title, X_val_link, X_val_prev_sentence])


for i in range(10,65):
    binary_predictions = (predictions > i/100).astype(int)
    f1 = f1_score(binary_predictions, y_test)
    print("Pour i=",i/100, " le score F1 est: ", f1)

482/482 [==============================] - 2s 4ms/step
Pour i= 0.1  le score F1 est:  0.49971214738054115
Pour i= 0.11  le score F1 est:  0.5058525165821304
Pour i= 0.12  le score F1 est:  0.5122096486003573
Pour i= 0.13  le score F1 est:  0.5181818181818182
Pour i= 0.14  le score F1 est:  0.5238828967642527
Pour i= 0.15  le score F1 est:  0.5291236844847349
Pour i= 0.16  le score F1 est:  0.5352978588085647
Pour i= 0.17  le score F1 est:  0.5401098309464843
Pour i= 0.18  le score F1 est:  0.5472484104363078
Pour i= 0.19  le score F1 est:  0.5526754922683279
Pour i= 0.2  le score F1 est:  0.5581131647579091
Pour i= 0.21  le score F1 est:  0.5622693726937269
Pour i= 0.22  le score F1 est:  0.5651151857075694
Pour i= 0.23  le score F1 est:  0.5698257340654095
Pour i= 0.24  le score F1 est:  0.5732669661284449
Pour i= 0.25  le score F1 est:  0.5760078905190482
Pour i= 0.26  le score F1 est:  0.5798266549428464
Pour i= 0.27  le score F1 est:  0.5821812596006145
Pour i= 0.28  le score F1 es

In [78]:
################################################ Kaggle data #########################################################


import json
from pathlib import Path
from sklearn.svm import SVR
from sentence_transformers import SentenceTransformer
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import f1_score, accuracy_score




def flatten(list_of_list):
    return [item for sublist in list_of_list for item in sublist]


path_to_test = Path("test")  #be careful it's changed to training here





########################################################## Validation Data #######################################################################


path_to_test = Path("test")  



#kaggle test data
test_set = ['ES2003', 'ES2004', 'ES2011', 'ES2014', 'IS1008', 'IS1009', 'TS3003', 'TS3004', 'TS3006', 'TS3007']
test_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in test_set])


# text_SVD: utterances are embedded with SentenceTransformer, then train a support vector regression model.
bert = SentenceTransformer('all-MiniLM-L6-v2')



test_labels = {}
test_labels2 = {}
test_labels3 = {}
for transcription_id in test_set:
    X_test = [] 
    X_test_edges = []
    X_test_speaker = []
    X_test_previous = []
    X_test_edges.append("Beginning")
    
    file_path = "test" + "/" + transcription_id + '.txt'
    # Open the file in read mode


    X_test_previous_number = []
    X_test_previous_number.append(0)



    with open(file_path, 'r') as file:
        edge_type_transcriptionId = []
        # Read each line of the file
        c_line = 0
        for line in file:
            # Split the line into words using space as a separator
            words = line.split()

            # If the line is not empty and contains at least three words
            if words:
                edge_type_transcriptionId.append(words[1])    # Type of the edge
            else:
                print("WORDSSSSSSSSSSSSSSSSSSSSSSS")

            
            X_test_previous_number.append(int(words[0]))

            c_line += 1
    

    with open(path_to_test / f"{transcription_id}.json", "r") as file:
        transcription = json.load(file)


    X_test_text_current = []



    c = 0
    for utterance in transcription:
        if(utterance["speaker"]  == "PM"):
            speaker = "Project Manager"
        elif(utterance["speaker"]  == "ME"):
            speaker = "Marketing Expert"
        elif(utterance["speaker"]  == "UI"):
            speaker = "Interface Designer"
        else:
            speaker = "Industrial Desginer"
        
        X_test_speaker.append(speaker)


        if(c == 0 ):
            X_test.append(utterance["text"])
            X_test_text_current.append(utterance["text"])
        else:
            X_test.append(utterance["text"])
            X_test_edges.append(edge_type_transcriptionId[c-1])
            X_test_text_current.append(utterance["text"])
        c+=1


    #print("X test text current" , X_test_text_current)
    X_test_previous.append("")
    for j in range(1, len(X_test_text_current)):
        X_test_previous.append(X_test_text_current[X_test_previous_number[j]])



    X_test = bert.encode(X_test, show_progress_bar=True)
    X_test_edges = bert.encode(X_test_edges, show_progress_bar=True)
    X_test_speaker = bert.encode(X_test_speaker, show_progress_bar=True)
    X_test_previous = bert.encode(X_test_previous, show_progress_bar=True)


    
    X_val_text = X_test 
    X_val_title = X_test_edges 
    X_val_link = X_test_speaker 
    X_val_prev_sentence = X_test_previous 



    X_val_text = np.array(X_val_text)
    X_val_title = np.array(X_val_title)
    X_val_link = np.array(X_val_link)
    X_val_prev_sentence = np.array(X_val_prev_sentence)


    threshold = 0.32
    y_test = model.predict([X_val_text, X_val_title, X_val_link, X_val_prev_sentence])
    binary_labels = [1 if pred > threshold else 0 for pred in y_test]

    test_labels[transcription_id] = binary_labels
    print(transcription_id)
    print(binary_labels)


    threshold = 0.34
    binary_labels2 = [1 if pred > threshold else 0 for pred in y_test]

    

    threshold = 0.36
    binary_labels3 = [1 if pred > threshold else 0 for pred in y_test]


    test_labels[transcription_id] = binary_labels
    test_labels2[transcription_id] = binary_labels2
    test_labels3[transcription_id] = binary_labels3

with open("NN_All_larger_dropout_t32_v1.json", "w") as file:
    json.dump(test_labels, file, indent=4)

with open("NN_All_larger_dropout_t34_v1.json", "w") as file:
    json.dump(test_labels3, file, indent=4)

with open("NN_All_larger_dropout_t36_v1.json", "w") as file:
    json.dump(test_labels3, file, indent=4)


Batches: 100%|██████████| 10/10 [00:01<00:00,  9.33it/s]

10/10 [==============================] - 0s 4ms/step


ES2003a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]


Batches: 100%|██████████| 24/24 [00:02<00:00,  8.64it/s]

 1/24 [>.............................] - ETA: 0s

...

 1/30 [>.............................] - ETA: 0s

30/30 [==============================] - 0s 5ms/step
TS3007c
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,

Batches: 100%|██████████| 46/46 [00:05<00:00,  7.88it/s]

 1/46 [..............................] - ETA: 1s

46/46 [==============================] - 0s 6ms/step
TS3007d
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,